## Librerias que utilizo

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importacion de la data y validaciones iniciales

In [4]:
titanic = pd.read_csv("titanic_MD.csv")
titanicG = pd.read_csv("titanic.csv")

print('Titanic Malo shape:', titanic.shape)
print('Titanic Bueno shape:', titanicG.shape)

Titanic Malo shape: (183, 12)
Titanic Bueno shape: (183, 12)


In [7]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",?,38.0,1.0,0.0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,NaN,1.0,NaN,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,NaN,0.0,113783,26.5500,C103,S


In [10]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,158.000000,180.000000,171.000000,175.000000
mean,455.366120,0.672131,1.191257,35.692532,0.461111,0.461988,78.959191
std,247.052476,0.470725,0.515187,15.640858,0.646122,0.753435,77.026328
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,35.500000,0.000000,0.000000,56.929200
75%,676.000000,1.000000,1.000000,48.000000,1.000000,1.000000,90.539600
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


## 1. Datos faltantes por columna

#### Buscando NAs

In [76]:
titanic.isna().sum()

PassengerId     0
Survived        0
Pclass          0
Name            0
Sex            51
Age            25
SibSp           3
Parch          12
Ticket          0
Fare            8
Cabin           0
Embarked       12
dtype: int64

#### Buscando valores extraños en valores String. De todos, solo Sex mostró valores diferentes fáciles de detectar

In [66]:
cols = []
val = []
for col in titanic.select_dtypes(include='object').columns:
    cols.append(col)
    val.append(titanic[col].str.contains(r'\?').sum())
pd.DataFrame({
    'cols':cols,
    'val':val
})

,cols,val
0,Name,0
1,Sex,51
2,Ticket,0
3,Cabin,0
4,Embarked,0


#### Buscando valores anormales en la data numérica. Age y Fare contienen valores extraños ya que Age no puede tener decimales y Fare no puede tener 0s

In [74]:
np.unique(titanic['Age'])

array([ 0.92,  2.  ,  3.  ,  4.  ,  6.  , 11.  , 14.  , 15.  , 16.  ,
       17.  , 18.  , 19.  , 21.  , 22.  , 23.  , 24.  , 25.  , 27.  ,
       28.  , 29.  , 30.  , 31.  , 32.  , 33.  , 34.  , 35.  , 36.  ,
       36.5 , 37.  , 38.  , 39.  , 40.  , 41.  , 42.  , 43.  , 44.  ,
       45.  , 46.  , 47.  , 48.  , 49.  , 50.  , 51.  , 52.  , 53.  ,
       54.  , 55.  , 56.  , 58.  , 60.  , 61.  , 62.  , 63.  , 64.  ,
       65.  , 70.  , 71.  , 80.  ,   nan])

In [54]:
titanic["Fare"].quantile([.00, .10, .25, .50, .75, .90, 1.00])

0.00      0.00000
0.10     13.31668
0.25     29.70000
0.50     56.92920
0.75     90.53960
0.90    153.46250
1.00    512.32920
Name: Fare, dtype: float64

### Survived, Pclass están bien. Sex, Age, SibSp, Parch, Fare y Embarked tienen datos faltantes detallados previamente.
### Se excluyeron de la exploración NAME, CABIN, PASSENGERID y TICKET, ya que estas cuatro columnas describen a una sola persona, no a un conjunto, por lo que a la hora de predecir si puede sobrevivir o no, estas columnas muy específicas no nos servirían

## 2. Tipo de limpieza de datos por columna

#### Sex (texto): usaré la Moda Sectorizada porque puedo sectorizar por valores que ya tengo para sacar cuál es el sexo que más se repite por sector y asumir que esos faltantes lo son, ya que 51 observaciones si afecta.
#### Age (número): usaré la Media porque el promedio de todos a bordo puede abarcar mejor el rango normal, aunque los valores extremos pueden fallar, los valores centrales de la distribución pueden estar muy cerca de la imputación.
#### SibSp (número): usaré la Moda porque son solo 3 datos faltantes y partiré por el supuesto de que esas 3 personas estaban casadas y tenian el mismo número de hijos por temas culturales de la época.
#### Parch (número): usaré la Media aproximada ya que son 12 datos, no afectará mucho y son solo valores enteros. Además, voy a partir del supuesto de que esas 12 personas tienen el promedio de familiares extra a bordo pero aproximado al valor más cercano.
#### Fare (número): usaré una Regresión Lineal ya que puedo usar la clase y valores faltantes calculados (como acompañantes) para predecir el posible valor de su ticket.
#### Embarked (texto): usaré la Moda porque son pocos datos faltantes (no influye mucho en el modelo) y partiré por el supuesto de que esas 12 personas abordaron en el mismo puerto.

## 3. Cantidad de filas completas

In [81]:
titanic.replace(r'\?', np.nan, regex = True, inplace = True)
titanic.loc[titanic.Fare<1] = np.nan
titanic.dropna().shape

(99, 12)

#### Hay 99 filas completas en la tabla, tomando las consideraciones descritas arriba